### Experiment5_LeedsDataset_Alltasks
Author : @leopauly | cnlp@leeds.ac.uk <br>
Description : Studying the feature vectors in the feature space

In [1]:
import os
from six.moves import xrange  # pylint: disable=redefined-builtin
import PIL.Image as Image
import random
import numpy as np
import cv2
import time
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from skimage import io

# Custom scripts
import lscript as lsp
import modelling as md

Using TensorFlow backend.


In [2]:
height=112 
width=112 
channel=3
crop_size=112

cluster_length=16
feature_size=8192 #4608 #   #16384  #487
nb_classes=2
saved_path='/nobackup/leopauly/S2l/'

batch_size=32
memory_batch_size_train=266
memory_batch_size_test=170
next_batch_start=0
sample_batch_size=10

####  Loading model and functions for calculating distance

In [3]:
## Defining placeholders in tf for images and targets
x_image = tf.placeholder(tf.float32, [None, 16,height,width,channel],name='x') 
y_true = tf.placeholder(tf.float32, [None, nb_classes],name='y_true')
y_true_cls = tf.placeholder(tf.int64, [None],name='y_true_cls')

model_keras = md.C3D_ucf101_training_model_tf(summary=True)
out=model_keras(x_image)
y_pred = tf.nn.softmax(out)
y_pred_cls = tf.argmax(out, dimension=1)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Miscellenious items finished..!!',flush=True)

/home/home01/cnlp/Seeing_to_Learn/Observation-Learning/ExperimentSet_RA-L/modelling.py:281: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", name="conv1", input_shape=(16, 112, ..., padding="same")`
  input_shape=input_shape))
/home/home01/cnlp/Seeing_to_Learn/Observation-Learning/ExperimentSet_RA-L/modelling.py:283: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
  border_mode='valid', name='pool1'))
/home/home01/cnlp/Seeing_to_Learn/Observation-Learning/ExperimentSet_RA-L/modelling.py:286: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), activation="relu", name="conv2", padding="same")`
  border_mode='same', name='conv2'))
/home/home01/cnlp/Seeing_to_Learn/Observation-Learning/ExperimentSet_RA-L/modelling.py:288: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_si

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 4, 14, 14, 256)    0         
__________

In [4]:
## Start the session with logging placement.
init_op = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
sess.run(init_op)

## Restore model weights from previously saved model
saver = tf.train.Saver()
saver.restore(sess, os.path.join(saved_path,'activity_model.ckpt-67'))
print("Model restored from file: %s" % saved_path,flush=True)

INFO:tensorflow:Restoring parameters from /nobackup/leopauly/S2l/activity_model.ckpt-67
Model restored from file: /nobackup/leopauly/S2l/


In [5]:
def get_compress_frames_data(filename, num_frames_per_clip=16):
  ''' Given a directory containing extracted frames, return a video clip of
  (num_frames_per_clip) consecutive frames as a list of np arrays '''
  ret_arr = []
  for parent, dirnames, filenames in os.walk(filename):

    filenames = sorted(filenames)
    jump=math.floor((len(filenames)/num_frames_per_clip))
    loop=0
    for i in range(0,len(filenames),jump):
      if (loop>15):
        break
      if (filenames[i].endswith('.png')):
        #print(str(filename) + '/' + str(filenames[i]))
        image_name = str(filename) + '/' + str(filenames[i])
        img = Image.open(image_name)
        img_data = np.array(img)
        ret_arr.append(img_data)
        loop=loop+1
  ret_arr=np.array(ret_arr)
  #ret_arr=ret_arr/255
  return np.array(ret_arr)

In [6]:
## Extraction of features 
def extract_video_features(vid):
    vid_=vid.reshape(-1,cluster_length,height,width,channel)
    f_v = sess.graph.get_tensor_by_name('flatten_1/Reshape:0')  #('dropout_2/cond/Merge:0')  #
    f_v_val=sess.run([f_v], feed_dict={'conv1_input:0':vid_,x_image:vid_,K.learning_phase(): 0 })
    features=np.reshape(f_v_val,(-1))
    return features

In [7]:
## Feature extraction from each class
def get_features_from_class(class_folder):
    feature_set_a=[]
    base_dir_a=class_folder
    sub_dir_a=os.listdir(base_dir_a)
    sub_dir_a=sorted(sub_dir_a)
    print(sub_dir_a)
    if '.DS_Store' in sub_dir_a:
        sub_dir_a.remove('.DS_Store')
    for sub_dir_a_ in sub_dir_a:
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        feature_set_a.append(extract_video_features(vid_a))
    return np.array(feature_set_a)

In [8]:
feature_set_a=get_features_from_class('/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Push/')
feature_set_b=get_features_from_class('/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/')
feature_set_c=get_features_from_class('/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach n Push/')
features_all=np.concatenate((feature_set_a,feature_set_b,feature_set_c),axis=0)
feature_labels_true=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2]

['.DS_Store', 'push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']
['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
['.DS_Store', 'Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2', 'multi_robo_sideview']


In [9]:
## Selecting top k features
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif
from sklearn.feature_selection import SelectKBest
selector= SelectKBest(f_classif, k=100)
selector.fit_transform(features_all,feature_labels_true)
select_feature_cols = selector.get_support(indices=True)
print(select_feature_cols)

def sel_feat(org_features):
    selected_features = [org_features[i] for i in select_feature_cols]
    return np.array(selected_features)

[  57  125  153  163  242  641  743  973 1016 1090 1171 1174 1207 1272 1302
 1304 1357 1383 1395 1548 1563 1573 1580 1614 1662 1689 1710 1747 1772 1780
 1791 1797 1800 1806 1808 1843 1895 1959 1987 1996 2004 2148 2286 2296 2342
 2347 2442 2499 2501 2584 2617 2626 2732 2799 2838 2853 3002 3169 3211 3360
 3576 3582 3620 3718 3786 3794 3807 3857 3973 4015 4077 4102 4112 4219 4410
 4447 4507 4531 4558 4600 4613 4624 4821 4886 5018 5256 5562 5595 5802 5839
 6343 6373 6467 6484 6615 6885 6970 7045 7131 7148]


/opt/conda/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [   1   30   38   46  144  155  159  265  352  379  389  390  521  671  777
  808  864  911 1006 1033 1170 1183 1289 1403 1545 1585 1603 1604 1613 1620
 1637 1640 1664 1683 1745 1771 1798 1801 1823 1831 1832 1856 1893 1915 1918
 1953 2009 2024 2162 2192 2203 2233 2319 2446 2465 2477 2525 2542 2829 2856
 2917 2932 2959 2989 3171 3191 3230 3471 3489 3501 3605 3606 3644 3651 3652
 3668 3676 3688 3771 3784 3793 3833 3846 3847 3865 3922 3989 4001 4011 4028
 4063 4072 4104 4121 4160 4242 4251 4254 4339 4365 4448 4510 4513 4522 4525
 4539 4570 4838 4877 4904 4960 4965 4980 4998 5007 5025 5052 5082 5129 5151
 5219 5239 5389 5519 5537 5641 5700 5724 5779 5928 5970 6004 6011 6049 6152
 6162 6169 6177 6195 6204 6218 6228 6254 6289 6295 6299 6329 6384 6387 6425
 6433 6440 6499 6542 6561 6562 6571 6599 6621 6674 6678 6681 6689 6748 6767
 6807 6809 6856 6945 6992 6994 7008 7019 7054 70

####  Feature extraction and claculating distances

In [10]:
## DISTANCE CALCULATION

dist_sim_class=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

## Calculating Distances
for sub_dir_a_ in sub_dir_a:
        print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=np.linalg.norm(sel_feat(extract_video_features(vid_a))-sel_feat(extract_video_features(vid_b)))
            dist_sim_class.append(feature_dist)

            
## Calculating intra-class variance
dist_sim_class_nozero=[x for x in dist_sim_class if x!=0]
intra_var_class=sum(dist_sim_class_nozero)/len(dist_sim_class_nozero)
print('Inter class variance bw class :',intra_var_class)

['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
reach_demo_0deg
reach_demo_0deg
reach_demo_180deg
reach_demo_green
reach_human
reach_robo
reach_robo_M2
reach_robo_M3
reach_robo_arbview1
reach_robo_bg
reach_robo_bg_change_target
reach_robo_change_pos
reach_robo_fast
reach_robo_obj2
reach_robo_sideview_new
reach_demo_180deg
reach_demo_0deg
reach_demo_180deg
reach_demo_green
reach_human
reach_robo
reach_robo_M2
reach_robo_M3
reach_robo_arbview1
reach_

In [11]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

for sub_dir_a_ in sub_dir_a:
        print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=np.linalg.norm(sel_feat(extract_video_features(vid_a))-sel_feat(extract_video_features(vid_b)))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('Inter class variance bw class 1 and class 2:',inter_var)
print('Shape',np.array(dist_dif).shape)

['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
['.DS_Store', 'push_demo_0deg', 'push_demo_180deg', 'push_demo_human', 'push_robo', 'push_robo_M2', 'push_robo_M3', 'push_robo_arbview1', 'push_robo_bg_fast', 'push_robo_change_pos', 'push_robo_changetarget', 'push_robo_fast', 'push_robo_green', 'push_robo_obj2_new', 'push_robo_sideview']
reach_demo_0deg
push_demo_0deg
push_demo_180deg
push_demo_human
push_robo
push_robo_M2
push_robo_M3
push_robo_arbview1
push_robo_bg_fast
push_robo_change_pos
push_robo_changetarget
push_robo_fast
push_robo_green
push_robo_obj2_new
push_robo_sideview
reach_demo_180deg
push_demo_0deg
push_demo_180deg
push_demo_human
push_robo
push_robo_M2
push_robo_M3
push_robo_arbview1
push_robo_bg_fast
push_robo_change_pos
pu

In [12]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach n Push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

for sub_dir_a_ in sub_dir_a:
        print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=np.linalg.norm(sel_feat(extract_video_features(vid_a))-sel_feat(extract_video_features(vid_b)))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('Inter class variance bw class 1 and class 2:',inter_var)
print('Shape',np.array(dist_dif).shape)

['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
['.DS_Store', 'Multi_robo_M3', 'multi_demo_0deg', 'multi_demo_180deg', 'multi_robo', 'multi_robo_M2', 'multi_robo_arbview1', 'multi_robo_bg', 'multi_robo_change_pos', 'multi_robo_change_target', 'multi_robo_fast_new', 'multi_robo_human_new', 'multi_robo_obj', 'multi_robo_obj2', 'multi_robo_sideview']
reach_demo_0deg
Multi_robo_M3
multi_demo_0deg
multi_demo_180deg
multi_robo
multi_robo_M2
multi_robo_arbview1
multi_robo_bg
multi_robo_change_pos
multi_robo_change_target
multi_robo_fast_new
multi_robo_human_new
multi_robo_obj
multi_robo_obj2
multi_robo_sideview
reach_demo_180deg
Multi_robo_M3
multi_demo_0deg
multi_demo_180deg
multi_robo
multi_robo_M2
multi_robo_arbview1
multi_robo_bg
multi_robo_ch

In [13]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Incomplete push/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

for sub_dir_a_ in sub_dir_a:
        print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=np.linalg.norm(sel_feat(extract_video_features(vid_a))-sel_feat(extract_video_features(vid_b)))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('Inter class variance bw class 1 and class 2:',inter_var)
print('Shape',np.array(dist_dif).shape)

['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
['.DS_Store', 'push_robo_incomp', 'push_robo_left', 'push_robo_misc', 'push_robo_out_new', 'push_robo_right']
reach_demo_0deg
push_robo_incomp
push_robo_left
push_robo_misc
push_robo_out_new
push_robo_right
reach_demo_180deg
push_robo_incomp
push_robo_left
push_robo_misc
push_robo_out_new
push_robo_right
reach_demo_green
push_robo_incomp
push_robo_left
push_robo_misc
push_robo_out_new
push_robo_right
reach_human
push_robo_incomp
push_robo_left
push_robo_misc
push_robo_out_new
push_robo_right
reach_robo
push_robo_incomp
push_robo_left
push_robo_misc
push_robo_out_new
push_robo_right
reach_robo_M2
push_robo_incomp
push_robo_left
push_robo_misc
push_robo_out_new
push_robo_right
reach_robo_M3
push

In [14]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Incomplete_reach/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

for sub_dir_a_ in sub_dir_a:
        print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=np.linalg.norm(sel_feat(extract_video_features(vid_a))-sel_feat(extract_video_features(vid_b)))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('Inter class variance bw class 1 and class 2:',inter_var)
print('Shape',np.array(dist_dif).shape)

['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
['.DS_Store', 'reach_robo_after_target', 'reach_robo_incomp', 'reach_robo_left', 'reach_robo_out', 'reach_robo_right']
reach_demo_0deg
reach_robo_after_target
reach_robo_incomp
reach_robo_left
reach_robo_out
reach_robo_right
reach_demo_180deg
reach_robo_after_target
reach_robo_incomp
reach_robo_left
reach_robo_out
reach_robo_right
reach_demo_green
reach_robo_after_target
reach_robo_incomp
reach_robo_left
reach_robo_out
reach_robo_right
reach_human
reach_robo_after_target
reach_robo_incomp
reach_robo_left
reach_robo_out
reach_robo_right
reach_robo
reach_robo_after_target
reach_robo_incomp
reach_robo_left
reach_robo_out
reach_robo_right
reach_robo_M2
reach_robo_after_target
reach_robo_incomp
rea

In [15]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Incomplete multi/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

for sub_dir_a_ in sub_dir_a:
        print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=np.linalg.norm(sel_feat(extract_video_features(vid_a))-sel_feat(extract_video_features(vid_b)))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('Inter class variance bw class 1 and class 2:',inter_var)
print('Shape',np.array(dist_dif).shape)

['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
['.DS_Store', 'multi_robo_incomp', 'multi_robo_left', 'multi_robo_misc', 'multi_robo_out', 'multi_robo_right']
reach_demo_0deg
multi_robo_incomp
multi_robo_left
multi_robo_misc
multi_robo_out
multi_robo_right
reach_demo_180deg
multi_robo_incomp
multi_robo_left
multi_robo_misc
multi_robo_out
multi_robo_right
reach_demo_green
multi_robo_incomp
multi_robo_left
multi_robo_misc
multi_robo_out
multi_robo_right
reach_human
multi_robo_incomp
multi_robo_left
multi_robo_misc
multi_robo_out
multi_robo_right
reach_robo
multi_robo_incomp
multi_robo_left
multi_robo_misc
multi_robo_out
multi_robo_right
reach_robo_M2
multi_robo_incomp
multi_robo_left
multi_robo_misc
multi_robo_out
multi_robo_right
reach_robo_

In [16]:
## Calculating distance

dist_dif=[]

base_dir_a='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Reach/'
sub_dir_a=os.listdir(base_dir_a)
sub_dir_a=sorted(sub_dir_a)
print(sub_dir_a)
if '.DS_Store' in sub_dir_a:
    sub_dir_a.remove('.DS_Store')

base_dir_b='/nobackup/leopauly/S2l/Dataset/Leeds_Dataset_ Only_tasks_CompvsIncomp/Incomplete/'
sub_dir_b=os.listdir(base_dir_b)
sub_dir_b=sorted(sub_dir_b)
print(sub_dir_b)
if '.DS_Store' in sub_dir_b:
    sub_dir_b.remove('.DS_Store')

for sub_dir_a_ in sub_dir_a:
        print(sub_dir_a_)
        vid_a=get_compress_frames_data(base_dir_a+sub_dir_a_)
        
        for sub_dir_b_ in sub_dir_b:
            print(sub_dir_b_)
            vid_b=get_compress_frames_data(base_dir_b+sub_dir_b_)
            feature_dist=np.linalg.norm(sel_feat(extract_video_features(vid_a))-sel_feat(extract_video_features(vid_b)))
            dist_dif.append(feature_dist)


## Calculating inter-class variance
inter_var=sum(dist_dif)/len(dist_dif)
print('Inter class variance bw class 1 and class 2:',inter_var)
print('Shape',np.array(dist_dif).shape)

['.DS_Store', 'reach_demo_0deg', 'reach_demo_180deg', 'reach_demo_green', 'reach_human', 'reach_robo', 'reach_robo_M2', 'reach_robo_M3', 'reach_robo_arbview1', 'reach_robo_bg', 'reach_robo_bg_change_target', 'reach_robo_change_pos', 'reach_robo_fast', 'reach_robo_obj2', 'reach_robo_sideview_new']
['.DS_Store', 'multi_robo_incomp', 'multi_robo_left', 'multi_robo_misc', 'multi_robo_out', 'multi_robo_right', 'push_robo_incomp', 'push_robo_left', 'push_robo_misc', 'push_robo_out_new', 'push_robo_right', 'reach_robo_after_target', 'reach_robo_incomp', 'reach_robo_left', 'reach_robo_out', 'reach_robo_right']
reach_demo_0deg
multi_robo_incomp
multi_robo_left
multi_robo_misc
multi_robo_out
multi_robo_right
push_robo_incomp
push_robo_left
push_robo_misc
push_robo_out_new
push_robo_right
reach_robo_after_target
reach_robo_incomp
reach_robo_left
reach_robo_out
reach_robo_right
reach_demo_180deg
multi_robo_incomp
multi_robo_left
multi_robo_misc
multi_robo_out
multi_robo_right
push_robo_incomp
push